<a href="https://colab.research.google.com/github/AntMigGF/plantcv/blob/master/CWSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CÁLCULO CWSI
CROP WATER STRESS INDEX

## BIBLIOTECAS

In [2]:
!pip install opencv-python

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import shapiro
from scipy.stats import pearsonr
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression
from IPython.display import display, clear_output

import ipywidgets as widgets
from ipywidgets import *

sns.set_style("darkgrid", {'font.family': 'serif', 'font.serif': ['Times New Roman']})

## IMPORTAR DATOS
Excel con datos ya filtrados (puede contener *outliers*)

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
df = pd.read_excel("/content/DATOS_filtrado_II.xlsx") #Lectura del archiv excel ya cargado

In [ ]:
df=df.dropna() #El código elimina los datos nulos

In [ ]:
df

### AJUSTES Y PREPARACIÓN PARA ELEMENTOS DINÁMICOS
Facilitan la visualización e interacción con el código

In [ ]:
#Asignar datos de fecha
fecha_mapping = {
    1: '10/03/2022',
    2: '26/05/2022',
    3: '04/10/2022',
    4: '30/03/2023',
    5: '29/06/2023',
    6: '16/05/2024',
    7: '17/05/2024',
    8: '03/06/2024',
    9: '06/06/2024'
}

df['Fecha'] = df['Fecha'].map(fecha_mapping)
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d/%m/%Y')

In [ ]:
# Crear un diccionario con los mapeos de los valores de 'Fecha' a las nuevas fechas
#variedad = {
   # 1: 'Picual',
   # 2: 'Arbequina'
#}

# Reemplazar los valores de 'Fecha' usando el diccionario y convertirlos al tipo de dato fecha
#df['Variedad'] = df['Variedad'].map(variedad)

In [ ]:
#Lista de columnas
possible_factors=list(df.columns)
print(possible_factors)

In [ ]:
grupo_selector = widgets.Dropdown(
    options=df.columns,
    description='Factor',
    disabled=False
)

valor_column_selector = widgets.Dropdown(
    options=[col for col in df.columns if df[col].dtype in ['float64', 'int64']],
    description='Valores',
    disabled=False
)

In [ ]:
factor_selector = widgets.Dropdown(
    options=possible_factors,
    description='Factor',
    disabled=False
)



---




## CÁLCULO DE PRESIONES DE VAPOR



*   eₐ ⇒ Presión de vapor a tª de aire
*   eₛ ⇒ Presión de vapor saturado a tª aire






In [ ]:

# Cálculo para presión de vapor saturado(es)
def saturated_vapor_pressure(temp):
    return 0.61078 * np.exp((17.27 * temp) / (temp + 237.3))


In [ ]:
df['es'] = df['Ta'].apply(saturated_vapor_pressure) #Crección de nueva columna y aplicación de la función

In [ ]:
df['ea']=(df['HR']/100)*df['es'] #Cálculo de presión de vapor a tª de aire

In [ ]:
df['DPV']=df['es']-df['ea'] #Diferencia de presión de vapor

In [ ]:
df



---



# ELIMINACIÓN DE OUTLIERS



---



In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

In [ ]:
df_filtrado = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]
df_filtrado



---



###### REVISAR -POSIBLE ELIMINACIÓN POR CÓDIGO MÁS SIMPLE EN CELDA ANTERIOR-

In [ ]:
# Función para calcular outliers por fecha
def calcular_outliers_por_fecha(df_fecha):
    Q1 = df_fecha['mean'].quantile(0.25)
    Q3 = df_fecha['mean'].quantile(0.75)
    RI = Q3 - Q1
    limite_inferior = Q1 - 1.5 * RI
    limite_superior = Q3 + 1.5 * RI

    # Identificar outliers para esta fecha
    df_fecha['es_outlier'] = (df_fecha['mean'] < limite_inferior) | (df_fecha['mean'] > limite_superior)

    return df_fecha


In [ ]:
# Aplicar la función calcular_outliers_por_fecha por cada fecha
df_out=df.groupby('Fecha').apply(calcular_outliers_por_fecha).reset_index(drop=True)

In [ ]:
df_out

In [ ]:
# Crear un diccionario con los mapeos de los valores de 'Fecha' a las nuevas fechas
fecha_mapping = {
    1: '10/03/2022',
    2: '26/05/2022',
    3: '04/10/2022',
    4: '30/03/2023',
    5: '29/06/2023',
    6: '16/05/2024',
    7: '17/05/2024',
    8: '03/06/2024',
    9: '06/06/2024'
}

# Reemplazar los valores de 'Fecha' usando el diccionario y convertirlos al tipo de dato fecha
df_out['Fecha'] = df_out['Fecha'].map(fecha_mapping)
df_out['Fecha'] = pd.to_datetime(df_out['Fecha'], format='%d/%m/%Y')

In [ ]:
outliers = df_out[df_out['es_outlier']]
outliers

In [ ]:
#Vemos cuantas anomalías tenemos por fecha
count_outliers = outliers.groupby('Fecha')['es_outlier'].sum()
count_outliers

##### ELIMINACIÓN DE OUTLIERS

In [ ]:
# Crear un diccionario con los mapeos de los valores de 'Fecha' a las nuevas fechas
variedad = {
    1: 'Picual',
    2: 'Arbequina'
}

# Reemplazar los valores de 'Fecha' usando el diccionario y convertirlos al tipo de dato fecha
df_filtrado['Variedad'] = df_filtrado['Variedad'].map(variedad)

In [ ]:
df_filtrado



---



# ANÁLISIS PARA LÍMITE INFERIOR EN LA RELACIÓN Tc-Ta CON DPV


#### GRÁFICA DE LA RELACIÓN ENTRE DPV Y Tc-Ta

In [ ]:
# Crear un gráfico de dispersión
plt.figure(figsize=(8, 6))
sns.scatterplot(data=df_filtrado, x='DPV', y='mean')
plt.title('Distribución de Tc_Ta vs. DPV')
plt.xlabel('DPV')
plt.ylabel('Tc_Ta')
plt.show()

In [ ]:
# Crear el gráfico de dispersión
plt.figure(figsize=(10, 6))
plt.scatter(df_filtrado['DPV'], df_filtrado['mean'])

# Anotar cada punto con la etiqueta de 'Fecha'
for index, row in df.iterrows():
    plt.annotate(row['Fecha'], (row['DPV'], row['mean']), textcoords="offset points", xytext=(0,10), ha='center')

# Etiquetas y título
plt.xlabel('DPV')
plt.ylabel('Tc_Ta')
plt.title('Distribución de datos entre Tc_Ta y DPV con etiquetas de Fecha')

# Mostrar el gráfico
plt.grid(True)
plt.show()

#### ANÁLISIS Y OBTENCIÓN DE REGRESIÓN LINEAL

Para obtener las *baselines* de manera experimental se debe de analizar si hay diferencia entre plantas con riego (manejo 1 y 2) y sin riego (manejo 3 y 4), de esta manera las *baselines* se toman como el límite superior para plantas totalmente estresadas (no tendría riego); como límite inferior se toman los datos de plantas con riego.

##### ANÁLISIS ESTADÍSTICO ENTRE MANEJOS DE RIEGO/RIEGO DEFICITARIO

Vemos si existen diferencias significativas mediante test de normalidad de Shaphiro-Wilk


In [ ]:
columna = 'mean'
stat, p_value = stats.shapiro(df_filtrado[columna])

print(f'Statistic: {stat}, p-value: {p_value}')
alpha = 0.05
if p_value > alpha:
    print(f'La muestra "{columna}" parece provenir de una distribución normal (no se rechaza H0)')
else:
    print(f'La muestra "{columna}" no parece provenir de una distribución normal (se rechaza H0)')

plt.figure(figsize=(10, 4))
sns.histplot(df_filtrado[columna], kde=True)
plt.title('Histograma de Tc-Ta')
plt.show()


Parece que Tc-Ta no siguen una distribución normal

Ahora se comprobará la normalidad para cada Manejo (se selecciona en el display inferior)

In [ ]:
output = widgets.Output()

def realizar_test_normalidad(factor): #Función test de normalidad
    with output:
        clear_output()
        if factor in df_filtrado.columns:
            grupos = df_filtrado.groupby(factor)

            resultados = {}
            for nombre_grupo, grupo in grupos:
                # Realizar el test de Kolmogorov-Smirnov para cada grupo
                stat, p_value = stats.kstest(grupo['mean'].dropna(), 'norm')
                resultados[nombre_grupo] = {'statistic': stat, 'p_value': p_value}

            # Mostrar los resultados
            for nombre_grupo, resultado in resultados.items():
                print(f'Grupo: {nombre_grupo}')
                print(f'Statistic: {resultado["statistic"]}, p-value: {resultado["p_value"]}')
                if resultado['p_value'] > 0.05:
                    print('Los datos PARECEN SEGUIR una distribución normal (no se rechaza H0)')
                else:
                    print('Los datos NO parecen seguir una distribución normal (se rechaza H0)')
                print('-'*50)
        else:
            print(f'El factor "{factor}" no se encuentra en las columnas del dataframe.')

def on_button_clicked(b): #Botón para ejecutar el test
    if factor_selector.value:
        realizar_test_normalidad(factor_selector.value)
    else:
        with output:
            clear_output()
            print('Por favor, selecciona un factor.')

def on_clear_button_clicked(b): #Botón para limpiar resultados
    with output:
        clear_output()
        print('Resultados limpiados.')

In [ ]:
# Crear botones para ejecutar el test y limpiar resultados
button = widgets.Button(description="Ejecutar Test de Normalidad")
button.on_click(on_button_clicked)

clear_button = widgets.Button(description="Limpiar Resultados")
clear_button.on_click(on_clear_button_clicked)

In [ ]:
display(factor_selector, button, clear_button, output)

Parece que en ningún tipo de manejo se sigue una distribución normal, así que se deberá emplear un análisis no paramétrico de los datos para ver si hay diferencias significativas entre los tipos de manejo

###### PRUEBA DE KRUSKAL-WALLIS



In [ ]:
# Realizar la prueba de Kruskal-Wallis
stat, p_value = stats.kruskal(
    df_filtrado[df_filtrado['Manejo'] == 1]['mean'],
    df_filtrado[df_filtrado['Manejo'] == 2]['mean'],
    df_filtrado[df_filtrado['Manejo'] == 3]['mean'],
    df_filtrado[df_filtrado['Manejo'] == 4]['mean']
)

# Mostrar los resultados
print(f'Estadístico de Kruskal-Wallis: {stat}')
print(f'Valor p: {p_value}')

# Interpretación del resultado
alpha = 0.05
if p_value < alpha:
    print('Hay diferencias significativas entre los grupos de Manejo.')
else:
    print('No hay diferencias significativas entre los grupos de Manejo.')

 Aunque en la metodología para NWSB (non-water stress baseline) se emplean los datos de plantas no sometidas a estrés hídrico, al no existir diferencias significativas, se van a elegir los datos por encima del tercer cuartil para el límite inferior. Al solo tener un dato de humedad relativa por fecha, los valores de



---



#### OBTENCIÓN DE NWSB (LÍMITE INFERIOR)

Se van a calcular por fecha

**Se obtienen los datos de cada fecha cuya Tc-Ta esté por encima del tercer cuartil**

In [ ]:
#Percentil elegido
percentil_alto = 75

In [ ]:
df_altos = df[df['mean'] > df_filtrado['mean'].quantile(percentil_alto / 100)]

In [ ]:
df_altos

**Se grafica la relación Tc-Ta con DPV**

In [ ]:
# Crear el scatterplot con Seaborn solo para los datos bajos
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_altos, x='DPV', y='mean', hue='Manejo', palette='viridis')

# Etiquetas y título
plt.xlabel('DPV')
plt.ylabel('Tc-Ta')
plt.title('Relación Tc-Ta vs. DPV')

# Mostrar el gráfico
plt.grid(True)
plt.show()

###### PARA CADA FECHA

In [ ]:
def calcular_regresion(df):
    # Preparar los datos
    X = df[['DPV']]  # Variable independiente
    y = df['mean']  # Variable dependiente

    # Ajustar el modelo
    modelo = LinearRegression()
    modelo.fit(X, y)

    # Extraer los parámetros
    pendiente = modelo.coef_[0]
    constante = modelo.intercept_

    return pd.Series({'pendiente': pendiente, 'constante': constante})



In [ ]:
# Agrupar por la columna 'Fecha' y aplicar la función para calcular regresión
resultados = df_altos.groupby('Fecha').apply(calcular_regresion).reset_index()

# Unir los resultados de regresión con el DataFrame original
df_altos = df_altos.merge(resultados, on='Fecha', how='left')



In [ ]:
# prompt: save 'df_altos' as excel document but in my PC

from google.colab import files

# Save the DataFrame to an Excel file
df_altos.to_excel('df_altos.xlsx', index=False)

# Download the file to your local machine
files.download('df_altos.xlsx')


In [ ]:
df_altos



---



In [ ]:
# Preparar los datos
X = df_altos[['DPV']]  # Variable independiente
y = df_altos['mean']  # Variable dependiente

# Ajustar el modelo
modelo = LinearRegression()
modelo.fit(X, y)

# Extraer los parámetros
pendiente = modelo.coef_[0]
constante = modelo.intercept_


In [ ]:
print(f'Pendiente: {pendiente}')
print(f'Constante: {constante}')

In [ ]:
df_altos['Twet']=constante+(pendiente*df_altos['DPV'])
